## Text Classification of Drug Effectiveness from Customer Review

#### Purpose: Predict the effectiveness Level of Drugs from drug takers' feedbacks (including benefit reviews and overall comments)

#### - Testing the Trained Best Model

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from time import time
from sklearn.metrics import classification_report,precision_recall_curve,auc,accuracy_score,f1_score, confusion_matrix, ConfusionMatrixDisplay
# disable chaining warning
pd.options.mode.chained_assignment = None
import pickle

#### Import Test Data

In [2]:
test_file = 'data/drugLibTest_raw.tsv'
test_raw = pd.read_csv(test_file,sep='\t')
print('Test data:')
test_raw.head(5)

Test data:


,Unnamed: 0,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,1366,biaxin,9,Considerably Effective,Mild Side Effects,sinus infection,The antibiotic may have destroyed bacteria cau...,"Some back pain, some nauseau.",Took the antibiotics for 14 days. Sinus infect...
1,3724,lamictal,9,Highly Effective,Mild Side Effects,bipolar disorder,Lamictal stabilized my serious mood swings. On...,"Drowsiness, a bit of mental numbness. If you t...",Severe mood swings between hypomania and depre...
2,3824,depakene,4,Moderately Effective,Severe Side Effects,bipolar disorder,Initial benefits were comparable to the brand ...,"Depakene has a very thin coating, which caused...",Depakote was prescribed to me by a Kaiser psyc...
3,969,sarafem,10,Highly Effective,No Side Effects,bi-polar / anxiety,It controlls my mood swings. It helps me think...,I didnt really notice any side effects.,This drug may not be for everyone but its wond...
4,696,accutane,10,Highly Effective,Mild Side Effects,nodular acne,Within one week of treatment superficial acne ...,Side effects included moderate to severe dry s...,Drug was taken in gelatin tablet at 0.5 mg per...


### Import Text Pre-processor

In [3]:
# import user-defined stopwordList for this dataset
import drugname
import conditionList
# import user-defined text processor
import text_processor

# import vectorizer for feature engineering
with open('vectorizer.pickle','rb') as input:
    vectorizer = pickle.load(input)
vectorizer

TfidfVectorizer(max_features=3000, min_df=3, token_pattern='\\w+|\\d|\\S+')

### Import Best model

In [4]:
# MLP
hidden_size = 40
input_size = vectorizer.max_features
num_classes = 3
activation = nn.ReLU()

import MLP_Net_layer # import user-defined MLP Class
mlp_mdl = MLP_Net_layer.MLP_Net_layer(input_size,hidden_size,num_classes,activation) # import user-defined MLP Class
mlp_mdl.load_state_dict(torch.load('MLP_Final_Model')) # import model parameters
print(mlp_mdl)
print(list(mlp_mdl.parameters()))

MLP_Net_layer(
  (layer_1): Linear(in_features=3000, out_features=40, bias=True)
  (activation1): ReLU()
  (output_layer): Linear(in_features=40, out_features=3, bias=True)
)
[Parameter containing:
tensor([[-0.1464, -0.2528, -0.2298,  ..., -0.3444,  0.1045,  0.0100],
        [ 0.1202,  0.1621,  0.1845,  ..., -0.2235,  0.0311,  0.1376],
        [-0.3572, -0.3511, -0.4115,  ..., -0.2961, -0.2929, -0.1124],
        ...,
        [ 0.1798,  0.2090,  0.2206,  ..., -0.2100, -0.0640,  0.2195],
        [ 0.1939,  0.2257,  0.2772,  ...,  0.2277,  0.2159, -0.0874],
        [-0.2658, -0.2631, -0.3220,  ..., -0.1153, -0.1829, -0.3405]],
       requires_grad=True), Parameter containing:
tensor([ 1.1346e-02,  1.1313e-01,  1.6654e-01,  1.1095e-01,  1.9393e-02,
         6.2988e-02,  7.2430e-03,  5.8212e-03,  1.2005e-01,  4.2679e-02,
         1.0735e-02,  8.5095e-03,  1.7788e-01,  3.5382e-02,  2.2607e-02,
         1.6330e-01, -6.9137e-03,  4.7267e-02,  1.4256e-01,  2.4812e-01,
         1.0786e-01,  1.79

In [5]:
# SVC
with open('SVC_Final_Model.pickle','rb') as model:
    svc_classifier = pickle.load(model)
print(svc_classifier)

OneVsRestClassifier(estimator=LinearSVC(C=0.121, random_state=123))


### Pre-process test data

In [6]:
print(test_raw.info())
test_raw.dropna(axis=0, how='any',inplace=True)
test_df = text_processor.text_preprocess(test_raw,vectorizer,'effectiveness','commentsReview','benefitsReview')
test_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036 entries, 0 to 1035
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         1036 non-null   int64 
 1   urlDrugName        1036 non-null   object
 2   rating             1036 non-null   int64 
 3   effectiveness      1036 non-null   object
 4   sideEffects        1036 non-null   object
 5   condition          1036 non-null   object
 6   benefitsReview     1036 non-null   object
 7   sideEffectsReview  1036 non-null   object
 8   commentsReview     1036 non-null   object
dtypes: int64(2), object(7)
memory usage: 73.0+ KB
None


,abate,abcess,abdomen,abdominal,ability,able,abnormal,abnormality,abovei,abruptly,...,young,youth,yr,zanaflex,zap,zero,zetia,zombie,zone,effectiveness_ordinal
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
1,0.0,0.0,0.0,0.0,0.0,0.056599,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


### Performance Metric on MLP Model

In [13]:
y_true = torch.from_numpy(np.array(test_df.effectiveness_ordinal)).type(torch.LongTensor)
X_test = torch.from_numpy(np.array(test_df.iloc[:,:-1])).type(torch.FloatTensor)
target_names = ['Ineffective',
 'Marginally Effective/ Moderately Effective','Highly Effective/ Considerably Effective']
label = [0,1,2]
t0= time()
mlp_output = mlp_mdl(X_test)
_,pred_mlp = torch.max(mlp_output,1)
runtime = time() - t0
correct_output_mlp = (pred_mlp.data==y_true.data).sum().item()
mlp_accuracy = correct_output_mlp / test_df.shape[0] *100
report = classification_report(y_true,pred_mlp.data, labels=label, target_names=target_names)
print(report)
print('Total running time: %d seconds' %runtime)
print('Test Accuracy: %.3f%%' %mlp_accuracy)


                                            precision    recall  f1-score   support

                               Ineffective       0.25      0.15      0.18        82
Marginally Effective/ Moderately Effective       0.43      0.16      0.24       233
  Highly Effective/ Considerably Effective       0.74      0.92      0.82       721

                                  accuracy                           0.69      1036
                                 macro avg       0.47      0.41      0.41      1036
                              weighted avg       0.63      0.69      0.64      1036

Total running time: 0 seconds
Test Accuracy: 68.726%


### Performance Metric on SVC Model

In [35]:
y_true = test_df.effectiveness_ordinal
t1 = time()
y_pred_svc = svc_classifier.predict(test_df.iloc[:,:-1])
runtime_svc = time() - t1        
# metric calculation
svc_accuracy = accuracy_score(y_true,y_pred_svc) 
f1_class0 = f1_score(y_true,y_pred_svc,average='macro',labels=[0],zero_division=0)
f1_class1 = f1_score(y_true,y_pred_svc,average='macro',labels=[1],zero_division=0)
f1_class2 = f1_score(y_true,y_pred_svc,average='macro',labels=[2],zero_division=0)
report = classification_report(y_true,y_pred_svc, labels=label, target_names=target_names)
print(report)
print('Total running time: %d seconds' %runtime_svc)
print('Test Accuracy: %.3f' %svc_accuracy)


                                            precision    recall  f1-score   support

                               Ineffective       0.35      0.35      0.35        82
Marginally Effective/ Moderately Effective       0.66      0.10      0.17       233
  Highly Effective/ Considerably Effective       0.75      0.95      0.84       721

                                  accuracy                           0.71      1036
                                 macro avg       0.58      0.47      0.45      1036
                              weighted avg       0.70      0.71      0.65      1036

Total running time: 0 seconds
Test Accuracy: 0.713
